## Connecting to the database
The following code is required for connecting Jupyter Notebook to our database file.

In [41]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Exploring the data
We print out the first five rows of the dataset to understand it firstly.

In [42]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


As seen above, the dataset contains area and population information about countries. 

- `name` - The name of the country.
- `area` - The total land and sea area of the country.
- `population` - The country's population.
- `population_growth`- The country's population growth as a percentage.
- `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
- `death_rate` - The country's death rate, or the number of death a year per 1,000 people.
- `area` - The country's total area (both land and water).
- `area_land` - The country's land area in square kilometers.
- `area_water` - The country's waterarea in square kilometers.

We have to write `%%sql` at the beginning of the each cell because being able to use SQL syntax.

We begin to examine the data by finding the maximum and minimum values of population columns, .

In [43]:
%%sql
SELECT  MIN(population) 'Lowest Population', 
        MAX(population) 'Highest Population', 
        MIN(population_growth) 'Lowest Pop. Growth', 
        MAX(population_growth) 'Highest Pop. Growth'
FROM facts

 * sqlite:///factbook.db
Done.


Lowest Population,Highest Population,Lowest Pop. Growth,Highest Pop. Growth
0,7256490011,0.0,4.02


It seems that minimum and maximum population valuee are weird. Normally, population of a country should not be 0. Population of the most crowded country is nearly equal total population of the world. We take a closer look at these values.

In [44]:
%%sql
SELECT *
FROM facts
WHERE population==(SELECT MIN(population)
                  FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


The country with 'zero' population is Antarctica. Antarctica is not a country actually. We understand that the dataset can also include non-country rows. Now let's investigate the most populous country.

In [45]:
%%sql
SELECT *
FROM facts
WHERE population==(SELECT MAX(population)
                  FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We also understand the reason why the highest population value is too high. It gives the total population of the world. 

Now let's recalculate the values without the row of the world.

In [46]:
%%sql
SELECT  MIN(population) 'Lowest Population', 
        MAX(population) 'Highest Population', 
        MIN(population_growth) 'Lowest Pop. Growth', 
        MAX(population_growth) 'Highest Pop. Growth'
FROM facts
WHERE name!="World"

 * sqlite:///factbook.db
Done.


Lowest Population,Highest Population,Lowest Pop. Growth,Highest Pop. Growth
0,1367485388,0.0,4.02


The population of the world's most populous country is about 1.4 billion. It is known that this country is China.

## Finding densely populated contries 

Let's find the countries with the higher population density. We need the average population and area of the countries for this goal.

In [47]:
%%sql
SELECT  AVG(population) 'Avg Pop', 
        AVG(area) 'Avg Area'
FROM facts
WHERE name!='World'

 * sqlite:///factbook.db
Done.


Avg Pop,Avg Area
32242666.56846473,555093.546184739


Countries with population above world average population and area below world average may be considered densely.

In [48]:
%%sql
SELECT *
FROM facts
WHERE population>(SELECT AVG(population) 
                  FROM facts
                  WHERE name!='World')
AND area<(SELECT AVG(area) 
          FROM facts
          WHERE name!='World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


The countries above are high-populated in a relatively small area.

Let's try to answer questions below.

### Which country has the most people? 

In [91]:
%%sql
SELECT  name, 
        MAX(population) AS Highest_Pop
FROM facts
WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


name,Highest_Pop
China,1367485388


### Which country has the highest growth rate?

In [92]:
%%sql
SELECT name, MAX(population_growth) AS Highest_Pop_Growth
FROM facts
WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


name,Highest_Pop_Growth
South Sudan,4.02


### Which countries have the highest ratios of water to land? 

In [51]:
%%sql
SELECT  name, 
        ROUND(CAST(area_water AS float)/area_land,2) AS water_land_ratio
FROM facts
WHERE name <> 'World'
ORDER BY water_land_ratio DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


British Indian Ocean Territory and Virgin Islands have more water-area than land-area.

### Which countries will add the most people to their populations next year?

In [95]:
%%sql
SELECT  name,
        population,
        ROUND(population_growth*population/100, 0) AS inc_in_population
FROM facts
WHERE name<>'World'
ORDER BY inc_in_population DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,population,inc_in_population
India,1251695584,15270686.0
China,1367485388,6153684.0
Nigeria,181562056,4448270.0
Pakistan,199085847,2906653.0
Ethiopia,99465819,2874562.0
Bangladesh,168957745,2703324.0
United States,321368864,2506677.0
Indonesia,255993674,2355142.0
"Congo, Democratic Republic of the",79375136,1944691.0
Philippines,100998376,1626074.0


As expected, the number of people to be added is high in large-populated countries.

### Which countries have a higher death rate than birth rate?

In [88]:
%%sql
SELECT  name,
        ROUND(birth_rate-death_rate, 2) AS growth_rate
FROM facts
WHERE death_rate>birth_rate
ORDER BY growth_rate

 * sqlite:///factbook.db
Done.


name,growth_rate
Bulgaria,-5.52
Serbia,-4.58
Latvia,-4.31
Lithuania,-4.17
Ukraine,-3.74
Hungary,-3.57
Germany,-2.95
Slovenia,-2.95
Romania,-2.76
Croatia,-2.73
